In [ ]:
import numpy as np

import bokeh.io
import bokeh.plotting

import scipy.integrate
import scipy.optimize

import biocircuits
import colorcet

import matplotlib.pyplot as plt

from bokeh.io import output_notebook, show



In [1]:
def toggle_nullclines():
    """Make an interactive plot of nullclines and fixed points of
    the Gardner-Collins synthetic toggle switch.
    """
    # Set up sliders
    params = [
        dict(
            name="βx",
            start=0.1,
            end=20,
            step=0.1,
            value=10,
            long_name="beta_x_slider",
        ),
        dict(
            name="βy",
            start=0.1,
            end=20,
            step=0.1,
            value=10,
            long_name="beta_y_slider",
        ),
        dict(name="n", start=1, end=10, step=0.1, value=4, long_name="n_slider"),
    ]
    sliders = [
        bokeh.models.Slider(
            start=param["start"],
            end=param["end"],
            value=param["value"],
            step=param["step"],
            title=param["name"],
            width=150,
        )
        for param in params
    ]

    # Build base plot with starting parameters
    beta = 10
    n = 4

    # Compute nullclines
    x_y = np.linspace(0, 20, 400)
    y_x = np.linspace(0, 20, 400)
    x_x = beta / (1 + y_x ** n)
    y_y = beta / (1 + x_y ** n)

    cds = bokeh.models.ColumnDataSource(data=dict(x_x=x_x, x_y=x_y, y_x=y_x, y_y=y_y))

    # Make the plot
    p = bokeh.plotting.figure(
        frame_height=250,
        frame_width=250,
        x_axis_label="x",
        y_axis_label="y",
        x_range=[-1, 20],
        y_range=[-1, 20],
    )
    p.line(x="x_x", y="y_x", source=cds, line_width=2, legend_label="x nullcline")
    p.line(
        x="x_y",
        y="y_y",
        source=cds,
        line_width=2,
        color="orange",
        legend_label="y nullcline",
    )
    cds_stable = bokeh.models.ColumnDataSource(
        dict(x=[0.0009999, 9.99999999999], y=[9.99999999999, 0.0009999])
    )
    cds_unstable = bokeh.models.ColumnDataSource(
        dict(x=[1.533012798623252], y=[1.533012798623252])
    )
    p.circle(source=cds_stable, x="x", y="y", color="black", size=10)
    p.circle(
        source=cds_unstable,
        x="x",
        y="y",
        line_color="black",
        fill_color="white",
        line_width=2,
        size=10,
    )

    # Callback (uses JavaScript)
    js_code = jsfuns["rootfinding"] + jsfuns["toggle_nullclines"] + "callback()"

    callback = bokeh.models.CustomJS(
        args=dict(cds=cds, cdsStable=cds_stable, cdsUnstable=cds_unstable), code=js_code
    )

    # We use the `js_on_change()` method to call the custom JavaScript code.
    for param, slider in zip(params, sliders):
        callback.args[param["long_name"]] = slider
        slider.js_on_change("value", callback)

    # Return layout
    return bokeh.layouts.row(
        p,
        bokeh.models.Spacer(width=30),
        bokeh.layouts.column(bokeh.models.Spacer(height=40), *sliders),
    )

In [10]:
# Render Bokeh plots inline
bokeh.io.output_notebook()

bokeh.io.show(biocircuits.jsplots.toggle_nullclines())

Loading BokehJS ...